In [1]:
import numpy as np
import pandas as pd

from math import *
from sklearn.cluster import DBSCAN

pd.options.display.max_rows = 5
pd.options.display.max_columns = 50

In [ ]:
l = ['0228693-200613084148143.csv', '0228696-200613084148143.csv', '0228699-200613084148143.csv', '0228685-200613084148143.csv', '0228689-200613084148143.csv', '0228687-200613084148143.csv']

df = pd.read_csv('Telegram Desktop\\' + l[0] , delimiter = '\t')
for i in range(1, len(l)):
    df = df.append(pd.read_csv('Telegram Desktop\\' + l[i], delimiter = '\t'))

df.head(10)

In [ ]:
def calc_dist(x1, y1, x2, y2):
    return sqrt((x1-x2)*(x1-x2) + (y1-y2)*(y1-y2))


def flight_by_bird(bird):
    a = [[]]
    
    #кластеры для каждого месяца
    for i in range(1, 13):
        df_one_species = df[(df['species'] == bird) & (df['month'] == i)][['decimalLatitude', 'decimalLongitude']]
        if df_one_species.shape[0] > 0:
            clustering = DBSCAN(eps=2, min_samples=20).fit(df_one_species)
            for j in range(clustering.labels_.max()+1):
                a.append([df_one_species[clustering.labels_ == j]['decimalLatitude'].mean(), df_one_species[clustering.labels_ == j]['decimalLongitude'].mean(), i])
    monthly_clusters = np.zeros((len(a) - 1, 5))
    monthly_clusters[:,:3] = np.array(a[1:])
    
    #кластеры кластеров
    clustering = DBSCAN(eps=5, min_samples=3).fit(monthly_clusters[:,:2])
    
    #переводим месяца в координаты (одна координата от января от июля, другая от октября до апреля)
    for i in range(monthly_clusters.shape[0]):
        month = monthly_clusters[i,2]
        if month <= 7:
            monthly_clusters[i,3] = month-1
        else:
            monthly_clusters[i,3] = 13-month
        if month <= 4:
            monthly_clusters[i,4] = month+2
        elif month <= 10:
            monthly_clusters[i,4] = 10-month
        else:
            monthly_clusters[i,4] = month-10
    
    #Средние координаты месяцев
    clusters = [[monthly_clusters[clustering.labels_ == j, 3].mean(), monthly_clusters[clustering.labels_ == j, 4].mean()] for j in range(clustering.labels_.max()+1)]
    
    #Находим 2 самых далёких по средему месяцу кластера
    first_cl = -1
    second_cl = -1
    time_dist = 0
    for i in range(len(clusters)):
        for j in range(i+1, len(clusters)):
            d = calc_dist(clusters[i][0], clusters[i][1], clusters[j][0], clusters[j][1])
            if (d > time_dist):
                time_dist = d
                first_cl = i
                second_cl = j
    
    #Если кластеры разнесены по времени, то перелётная, инчае нет. Для перелётных возвращаем серединки 2 кластеров, найденных в предыдущем пунке, для не перелётных 2 раза середину по всем
    if time_dist >= 2.5:
        return
            bird,
            'Перелётная',
            monthly_clusters[clustering.labels_ == first_cl, 0].mean(),
            monthly_clusters[clustering.labels_ == first_cl, 1].mean(),
            monthly_clusters[clustering.labels_ == second_cl, 0].mean(),
            monthly_clusters[clustering.labels_ == second_cl, 1].mean()
    else:
        return
            bird,
            'Не перелётная',
            monthly_clusters[:,0].mean(),
            monthly_clusters[:,1].mean(),
            monthly_clusters[:,0].mean(),
            monthly_clusters[:,1].mean()

In [ ]:
df1 = df.groupby('species')[['gbifID', 'individualCount']].agg({'gbifID' : 'count', 'individualCount' : 'sum'})
species = list(df1[df1['gbifID'] > 1000].index)

out = [[]]

for bird in species:
    r = flight_by_bird(bird)
    if (r != None):
        out.append(list(r))
    print(len(out)-1)

out = out[1:]
out

In [ ]:
df_out = pd.DataFrame(np.array(out), columns = ['Name', 'Type', 'Start_Latitude', 'Start_Longitude', 'Finish_Latitude', 'Finish_Longitude'])

In [ ]:
df_out.to_excel('birds.xlsx', index=False)